# Computable Diagrams with `jupyterlab-drawio` 

> _ft. [drawio](https://github.com/jgraph/drawio/), [drawio-export](https://github.com/jgraph/draw-image-export2), [puppeteer](https://github.com/puppeteer/puppeteer), and a cast of thousands_

| pull request | demo branch | binder demo | 
|:--:|------|------|
| [#65][pr] |  [@bollwyvl/jupyterlab-drawio][repo] | [![][badge]][binder] |

[pr]: https://github.com/QuantStack/jupyterlab-drawio/pull/65
[repo]: https://github.com/bollwyvl/jupyterlab-drawio/tree/add-drawio-export
[badge]: https://mybinder.org/static/images/badge_logo.svg
[binder]: https://gke.mybinder.org/v2/gh/bollwyvl/jupyterlab-drawio/add-drawio-export?urlpath=lab/tree/Computable+Diagrams.ipynb

In addition to the live editing experience (see [how it works](./testfiles/How%20it%20works.dio)), the drawio XML format can be created by other tools. Output formats, e.g. SVG, can also be used by different tools.

## Before we begin

Start the export server. It will try to install its dependencies, and requires `nodejs`.
```bash
!python scripts/drawio_export_demo.py
```

In [26]:
import requests, IPython.display as D, pathlib as P, networkx as nx, jinja2
from graphviz2drawio import graphviz2drawio

## Load a drawio XML file

In [27]:
dio = next(P.Path().glob("testfiles/How it works.dio"))

In [28]:
### it's really not very pretty
# D.Markdown(f"```xml\n{dio.read_text()}\n```")

## Make the request params

`drawio-export` partially documents the list of [URL request parameters](https://github.com/jgraph/draw-image-export2#common-parameters) which it accepts. For example, it mentions visible layers... without defining the format. It may be the same as the message format, mostly captured in the [editor widget](./src/editor.ts)

In [29]:
default_params = dict(
    format="pdf",
    xml=dio.read_text(),
    allPages="1",
    base64="1"
)

## Make a request

Happily, the output appears to be stable given the same input. Hooray!

In [30]:
def get_pdf(**params):
    final_params = dict(default_params)
    final_params.update(params)
    res = requests.post("http://localhost:8000", final_params)
    return res.text

In [31]:
def show_pdf(**params):
    pdf_text = get_pdf(**params) 
    return D.HTML(f"""
        <iframe src="data:application/pdf;base64,{pdf_text}" width="100%" height="600px"></iframe>
        """)

show_pdf()

# Generating drawio XML

The only thing I've found that can generate drawio is [graphviz2drawio](https://github.com/hbmartin/graphviz2drawio). So `.dot` it is!

> _...see also: [jupyterlab_graphviz](https://www.npmjs.com/package/@deathbeds/jupyterlab_graphviz)_

In [32]:
some_dot = """
    digraph g {
       dot -> graphviz -> {svg  png};
       {drawioxml svg png} -> drawio -> {svg png  html drawioxml};
       inkscape -> {svg png};
       {svg pdf} -> inkscape;
       svg -> drawioxml;
       drawio_export[label="drawio-export"];
       {drawioxml} -> drawio_export -> {pdf png};
    }
"""

In [33]:
def show_pdf_from_dot(dot=some_dot, **params):
    return show_pdf(xml=graphviz2drawio.convert(dot), **params)
show_pdf_from_dot()

## Things that speak `dot`

## networkx

In [34]:
a_graph = nx.drawing.nx_agraph.to_agraph(nx.generators.atlas.graph_atlas(42))

In [35]:
def show_pdf_from_networkx(graph=a_graph, **params):
    return show_pdf_from_dot(str(graph))
show_pdf_from_networkx()

### Jinja
`dot` is a simple, mostly-forgiving language.

In [36]:
tmpl = jinja2.Template("""
graph g {
    layout=circo
    {% for i in range(n) %}
    node_{{i}};
    {% endfor %}
    {% for i in range(n) %}
    node_{{i}} -- {
        node_{{ i % (n // m) }};
        {% if i %}
        node_{{ i - 1 }}
        {% endif %}
    }
    {% endfor %}
}
""")


In [37]:
show_pdf_from_dot(tmpl.render(n=20, m=7))